## Q1

Refer to q1.sql and q1.png

## Q2

In [7]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

import pandas as pd

appName = "Big Data Analytics"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
.set('spark.driver.host','127.0.0.1')\
.setAppName(appName)\
.setMaster(master)

# Create Spark Context 
sc = SparkContext.getOrCreate(conf=conf)

# Create SQL Context to conduct some database operations
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = sqlContext.sparkSession.builder\
.config("spark.jars.packages", "org.postgresql:postgresql:42.2.29") \
.config("spark.sql.execution.arrow.enabled","true")\
.getOrCreate() 

# spark.conf.set("spark.sql.execution.arrow.enabled","true")
# spark.conf.set("spark.jars.packages", "org.postgresql:postgresql:42.2.29")

In [8]:
import requests
from bs4 import BeautifulSoup

# Extract the html data -> pandasDF -> sparkDF
url = "http://news.google.com/rss/search?q=technology&hl=en-US&gl=US&ceid=US:en"
page = requests.get(url)

soup = BeautifulSoup(page.content, 'lxml')
tbl = soup.find_all('item')

/var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/ipykernel_95553/1797659306.py:8: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(page.content, 'lxml')


In [9]:
tbl[0]

<item><title>China Asks Its Carmakers to Keep Key EV Technology at Home - Bloomberg</title><link/>https://news.google.com/rss/articles/CBMirgFBVV95cUxNemdjcmNYY1ktLUhjTXBId1RzSG93ekhpOTFGblVnNUp1ZHE4LTV0SWhKQ1E5UmlDQmZUVlhpWWJoUjhJdnlXRmlJbVk2WFhPN1FxYjQ0U1RfV3ZJUm14U2FoSmNFbzByQW1sR2xPY25rZE1HejAycWF5WW1rNnpoVk0zNUk5NFd0d0g3MVJmeS1hb0tpMTQyaFcwdU0yYVJOUHB4TUt2dHQ1bmduZ0E?oc=5<guid ispermalink="false">CBMirgFBVV95cUxNemdjcmNYY1ktLUhjTXBId1RzSG93ekhpOTFGblVnNUp1ZHE4LTV0SWhKQ1E5UmlDQmZUVlhpWWJoUjhJdnlXRmlJbVk2WFhPN1FxYjQ0U1RfV3ZJUm14U2FoSmNFbzByQW1sR2xPY25rZE1HejAycWF5WW1rNnpoVk0zNUk5NFd0d0g3MVJmeS1hb0tpMTQyaFcwdU0yYVJOUHB4TUt2dHQ1bmduZ0E</guid><pubdate>Thu, 12 Sep 2024 03:50:00 GMT</pubdate><description>&lt;a href="https://news.google.com/rss/articles/CBMirgFBVV95cUxNemdjcmNYY1ktLUhjTXBId1RzSG93ekhpOTFGblVnNUp1ZHE4LTV0SWhKQ1E5UmlDQmZUVlhpWWJoUjhJdnlXRmlJbVk2WFhPN1FxYjQ0U1RfV3ZJUm14U2FoSmNFbzByQW1sR2xPY25rZE1HejAycWF5WW1rNnpoVk0zNUk5NFd0d0g3MVJmeS1hb0tpMTQyaFcwdU0yYVJOUHB4TUt2dHQ1bmduZ0E

In [10]:
def extract_link(test_str,sub1,sub2):
    # getting index of substrings
    idx1 = test_str.index(sub1)
    idx2 = test_str.index(sub2)

    res = ''
    # getting elements in between
    for idx in range(idx1 + len(sub1) + 1, idx2):
        res = res + test_str[idx]
    return res

In [11]:
data = []
col_name = ['title','link','pub_date','description','source']
for table in tbl:
    title = table.find('title', recursive=False).string
    link = extract_link(table.prettify(formatter="html"),'<link/>\n','\n <guid')
    pub_date = table.find('pubdate', recursive=False).string
    desc = extract_link(table.find("description").string,'target="_blank"','</a>')
    src = table.find('source', recursive=False).string
    data.append([title, link, pub_date, desc, src])

pandasDF = pd.DataFrame(data, columns=col_name)

In [12]:
#Create PySpark DataFrame from Pandas
spark.conf.set("spark.sql.execution.arrow.enabled","true")
sparkDF = spark.createDataFrame(pandasDF) 
sparkDF.printSchema()
sparkDF.show()

24/09/12 10:47:19 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


root
 |-- title: string (nullable = true)
 |-- link: string (nullable = true)
 |-- pub_date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|                link|            pub_date|         description|              source|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|China Asks Its Ca...|https://news.goog...|Thu, 12 Sep 2024 ...|China Asks Its Ca...|           Bloomberg|
|A new solution to...|https://news.goog...|Wed, 11 Sep 2024 ...|A new solution to...|   WCPO 9 Cincinnati|
|China Harnesses a...|https://news.goog...|Mon, 09 Sep 2024 ...|China Harnesses a...|The Wall Street J...|
|Students are buil...|https://news.goog...|Thu, 12 Sep 2024 ...|Students are buil...|                 CNN|
|Adobe, Soundcloud...|https://news.goog...|Tue, 10 

### Database Configuration

In [13]:
# Config the db properties
db_properties = {}
db_properties['username'] = "postgres"
db_properties['password'] = ""
db_properties['url'] = "jdbc:postgresql://localhost:5432/postgres"
db_properties['table'] = "news"
db_properties['driver'] = "org.postgresql.Driver"

In [14]:
# Insert the data from sparkDF into the db
sparkDF.write.format("jdbc").mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

In [15]:
# Test that the db has data
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", db_properties['url']) \
    .option("dbtable", db_properties['table']) \
    .option("user", db_properties['username']) \
    .option("password", db_properties['password']) \
    .option("driver", db_properties['driver']) \
    .load()

jdbcDF.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|                link|            pub_date|         description|              source|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|China Asks Its Ca...|https://news.goog...|Thu, 12 Sep 2024 ...|China Asks Its Ca...|           Bloomberg|
|A new solution to...|https://news.goog...|Wed, 11 Sep 2024 ...|A new solution to...|   WCPO 9 Cincinnati|
|China Harnesses a...|https://news.goog...|Mon, 09 Sep 2024 ...|China Harnesses a...|The Wall Street J...|
|Students are buil...|https://news.goog...|Thu, 12 Sep 2024 ...|Students are buil...|                 CNN|
|Adobe, Soundcloud...|https://news.goog...|Tue, 10 Sep 2024 ...|Adobe, Soundcloud...|             Digiday|
|A new dawn for th...|https://news.goog...|Wed, 11 Sep 2024 ...|A new dawn for th...|            McKinsey|
|Deadline for anti...|https://news.go

Refer to q2.png

## Q3

In [16]:
from datetime import datetime, timedelta
from pyspark.sql import functions as F

# Create a function to get the news that published in the last 24 hours
def get_last_24_hours_news(df):
    # Find the time boundary
    last_hours = datetime.now() - timedelta(hours = 24)
    print("The last 24 hours: ",last_hours)
    
    return df.filter(F.to_timestamp(F.regexp_replace("pub_date", r'\w{3},\s|\sGMT', ""), "dd MMM yyyy HH:mm:ss") > last_hours).orderBy("pub_date")

In [17]:
last_24_hours_news = get_last_24_hours_news(jdbcDF)
last_24_hours_news.show()

The last 24 hours:  2024-09-11 10:47:25.254918
+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|                link|            pub_date|         description|              source|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|China Asks Its Ca...|https://news.goog...|Thu, 12 Sep 2024 ...|China Asks Its Ca...|           Bloomberg|
|List of Medical D...|https://news.goog...|Thu, 12 Sep 2024 ...|List of Medical D...|       In Compliance|
|The World's First...|https://news.goog...|Thu, 12 Sep 2024 ...|The World's First...|         PR Newswire|
|Application detec...|https://news.goog...|Thu, 12 Sep 2024 ...|Application detec...|          CSO Online|
|China Wants Autom...|https://news.goog...|Thu, 12 Sep 2024 ...|China Wants Autom...|            Benzinga|
|Pennsylvania Coll...|https://news.goog...|Thu, 12 Sep 2024 ...|Pennsylvania Coll...|Williamsport

Refer to q3.png

## Q4

Refer to q4.sql

In [18]:
import requests
from bs4 import BeautifulSoup

# Link extraction function
def extract_link(test_str,sub1,sub2):
    # Getting index of substrings
    idx1 = test_str.index(sub1)
    idx2 = test_str.index(sub2)

    res = ''
    # Getting elements in between
    for idx in range(idx1 + len(sub1) + 1, idx2):
        res = res + test_str[idx]
    return res

# Populate data from the URL function
def populate_data(url):
    # Request to get the html data
    page = requests.get(url)

    # Extract the html to pandasDF
    soup = BeautifulSoup(page.content, 'lxml')
    tbl = soup.find_all('item')
    # tbl[0]

    # Create a new pandasDF
    data = []
    col_name = ['title','link','pub_date','description','source']
    for table in tbl:
        title = table.find('title', recursive=False).string
        link = extract_link(table.prettify(formatter="html"),'<link/>\n','\n <guid')
        pub_date = table.find('pubdate', recursive=False).string
        desc = extract_link(table.find("description").string,'target="_blank"','</a>')
        src = table.find('source', recursive=False).string
        data.append([title, link, pub_date, desc, src])
    
    pandasDF = pd.DataFrame(data, columns=col_name)
    return pandasDF

# Convert pandasDF to sparkDF function
def pandasDF_to_sparkDF(pandasDF):
    sparkDF = spark.createDataFrame(pandasDF) 
    sparkDF.printSchema()
    sparkDF.show()
    return sparkDF
    

In [19]:
# Populate Business and Sports news
busi_url = "https://news.google.com/rss/search?q=business&hl=en-US&gl=US&ceid=US:en"
sports_url = "https://news.google.com/rss/search?q=sports&hl=en-US&gl=US&ceid=US:en"

In [20]:
# Insert the business data into the db

# Extract the data from the html
busiDF = populate_data(busi_url)
busiDF.loc[:, "category"] = "business"
# Convert the business pandasDF to sparkDF
busi_sparkDF = pandasDF_to_sparkDF(busiDF)
# Insert into the db
busi_sparkDF.write.format("jdbc").mode("append")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

root
 |-- title: string (nullable = true)
 |-- link: string (nullable = true)
 |-- pub_date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|               title|                link|            pub_date|         description|              source|category|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|US businesses' op...|https://news.goog...|Thu, 12 Sep 2024 ...|US businesses' op...|             Reuters|business|
|7 ChatGPT Prompts...|https://news.goog...|Wed, 11 Sep 2024 ...|7 ChatGPT Prompts...|              Forbes|business|
|Why MrBeast could...|https://news.goog...|Thu, 12 Sep 2024 ...|Why MrBeast could...|    Business Insider|business|
|AI’s Language Lea...|https://news.goog...|Thu, 12 Sep 2024 ...|

/var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/ipykernel_95553/4156682966.py:22: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(page.content, 'lxml')


In [21]:
# Insert the soprts data into the db

# Extract the data from the html
sportsDF = populate_data(sports_url)
sportsDF.loc[:, "category"] = "sports"
# Convert the soprts pandasDF to sparkDF
sports_sparkDF = pandasDF_to_sparkDF(sportsDF)
# Insert into the db
sports_sparkDF.write.format("jdbc").mode("append")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

root
 |-- title: string (nullable = true)
 |-- link: string (nullable = true)
 |-- pub_date: string (nullable = true)
 |-- description: string (nullable = true)
 |-- source: string (nullable = true)
 |-- category: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|               title|                link|            pub_date|         description|              source|category|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+
|Sources: Pac-12 r...|https://news.goog...|Thu, 12 Sep 2024 ...|Sources: Pac-12 r...|        Yahoo Sports|  sports|
|Is MLB's RSN mess...|https://news.goog...|Wed, 11 Sep 2024 ...|Is MLB's RSN mess...|                ESPN|  sports|
|Report: Optimism ...|https://news.goog...|Thu, 12 Sep 2024 ...|Report: Optimism ...|          NBC Sports|  sports|
|Lawsuit Against N...|https://news.goog...|Thu, 12 Sep 2024 ...|

/var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/ipykernel_95553/4156682966.py:22: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(page.content, 'lxml')


In [22]:
# Get the data from the db
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", db_properties['url']) \
    .option("dbtable", db_properties['table']) \
    .option("user", db_properties['username']) \
    .option("password", db_properties['password']) \
    .option("driver", db_properties['driver']) \
    .load()

jdbcDF.groupby("category").count().show()

+----------+-----+
|  category|count|
+----------+-----+
|technology|  100|
|    sports|  100|
|  business|  100|
+----------+-----+



Refer to q4.png

## Q5

In [28]:
# Get the data from the db
jdbcDF = spark.read \
    .format("jdbc") \
    .option("url", db_properties['url']) \
    .option("dbtable", db_properties['table']) \
    .option("user", db_properties['username']) \
    .option("password", db_properties['password']) \
    .option("driver", db_properties['driver']) \
    .load()

jdbcDF.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|                link|            pub_date|         description|              source|  category|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|China Asks Its Ca...|https://news.goog...|Thu, 12 Sep 2024 ...|China Asks Its Ca...|           Bloomberg|technology|
|A new solution to...|https://news.goog...|Wed, 11 Sep 2024 ...|A new solution to...|   WCPO 9 Cincinnati|technology|
|China Harnesses a...|https://news.goog...|Mon, 09 Sep 2024 ...|China Harnesses a...|The Wall Street J...|technology|
|Students are buil...|https://news.goog...|Thu, 12 Sep 2024 ...|Students are buil...|                 CNN|technology|
|Adobe, Soundcloud...|https://news.goog...|Tue, 10 Sep 2024 ...|Adobe, Soundcloud...|             Digiday|technology|
|A new dawn for th...|https://news.goog...|Wed, 11 Sep 2

In [29]:
no_NLF_jdbcDF = jdbcDF.filter(~jdbcDF.title.like("%NFL%"))

In [24]:
no_NLF_jdbcDF.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|                link|            pub_date|         description|              source|  category|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|China Asks Its Ca...|https://news.goog...|Thu, 12 Sep 2024 ...|China Asks Its Ca...|           Bloomberg|technology|
|A new solution to...|https://news.goog...|Wed, 11 Sep 2024 ...|A new solution to...|   WCPO 9 Cincinnati|technology|
|China Harnesses a...|https://news.goog...|Mon, 09 Sep 2024 ...|China Harnesses a...|The Wall Street J...|technology|
|Students are buil...|https://news.goog...|Thu, 12 Sep 2024 ...|Students are buil...|                 CNN|technology|
|Adobe, Soundcloud...|https://news.goog...|Tue, 10 Sep 2024 ...|Adobe, Soundcloud...|             Digiday|technology|
|A new dawn for th...|https://news.goog...|Wed, 11 Sep 2

In [33]:
# Need to do cache to solve empty inserted data to the db
no_NLF_jdbcDF.cache()

24/09/12 10:53:30 WARN CacheManager: Asked to cache already cached data.


DataFrame[title: string, link: string, pub_date: string, description: string, source: string, category: string]

In [31]:
# Activate the cache() function with any functions
no_NLF_jdbcDF.show() 

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|                link|            pub_date|         description|              source|  category|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|China Asks Its Ca...|https://news.goog...|Thu, 12 Sep 2024 ...|China Asks Its Ca...|           Bloomberg|technology|
|A new solution to...|https://news.goog...|Wed, 11 Sep 2024 ...|A new solution to...|   WCPO 9 Cincinnati|technology|
|China Harnesses a...|https://news.goog...|Mon, 09 Sep 2024 ...|China Harnesses a...|The Wall Street J...|technology|
|Students are buil...|https://news.goog...|Thu, 12 Sep 2024 ...|Students are buil...|                 CNN|technology|
|Adobe, Soundcloud...|https://news.goog...|Tue, 10 Sep 2024 ...|Adobe, Soundcloud...|             Digiday|technology|
|A new dawn for th...|https://news.goog...|Wed, 11 Sep 2

In [32]:
# Update into the db
no_NLF_jdbcDF.write.format("jdbc").mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

Refer to q5.png

## Q6

Refer to q6.png